In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [2]:
from dotenv import load_dotenv
load_dotenv()

import os
bearer_token = os.environ.get("TOKEN")

In [3]:
os.environ['TOKEN'] = "AAAAAAAAAAAAAAAAAAAAABaiVAEAAAAA%2FakBYpnMJAbQFkU%2BBEb4fVBxU7c%3DdSU24Vlpg8VXNSzchoEMEt1jGeJRZACbVaIkXWlguvpZVQBNe8"

In [4]:
def auth():
    return os.environ.get("TOKEN")

In [5]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

In [6]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [7]:
def create_url(keyword, start_date, end_date, next_token):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword, 
                    'start_time' : start_list[i], 
                    'end_time' : end_list[i],
                    'max_results' : max_results,
                    'tweet.fields': 'created_at',
                    'next_token': {}
                     }
    
    return (search_url, query_params)

In [8]:
def connect_to_endpoint(url, headers, params, next_token):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.get( url, auth=bearer_oauth, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [9]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        #Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        #Tweet ID
        tweet_id = tweet['id']

        #Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [created_at, tweet_id, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter)

In [10]:
time_range = pd.read_csv("Twitter_start_end_times.csv")
start_list = time_range['start_time'].tolist()
end_list = time_range['end_time'].tolist()

In [11]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "tesla lang:en -from:tesla -is:retweet"
start_list = start_list
end_list = end_list

max_results = 100

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['created_at','id','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 200 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i], end_list[i], next_token)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7h0ms4kvsrug18cyiol3vwhz0d
Start Date:  2021-10-30T00:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  100
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv7h0ms4kvsrug18cyiol3vwhz0d
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7h0ms30k0yleedsmmagffuaxkt
Start Date:  2021-10-30T00:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  200
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7h0ndjc2vs1zkfg0a66c6yujjx
Start Date:  2021-10-30T01:00:00.000Z
# of Tweets added from this response:  98
Total # of Tweets added:  298
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv7h0ndjc2vs1zkfg0a66c6yujjx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7h0ndgbwiwi2fjelrp1c8psltp
Start Date:  2021-10-30T01:00:00.000Z
# of Twe

-------------------
Token:  b26v89c19zqg8o3fpdv7h4w8aoarlg4kw7gug79vqge4d
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7h4w894edwei214kmldjv3b919
Start Date:  2021-10-30T13:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  3094
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7h4wtnxpze4hjysa4zxor2qjct
Start Date:  2021-10-30T14:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  3194
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv7h4wtnxpze4hjysa4zxor2qjct
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7h4wtme921h9lyyxp5kule1if1
Start Date:  2021-10-30T14:00:00.000Z
# of Tweets added from this response:  99
Total # of Tweets added:  3293
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv7h4wtme921h9lyyxp5kule1if1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7h4wj7mh39ki1wkccembbmmohp
St

-------------------
Token:  b26v89c19zqg8o3fpdv7h95dxgps9qkkpmp1w366bppq5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7h95duf1iakc2q629imbd5rh4t
Start Date:  2021-10-31T02:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  6087
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7h95zf95jl0lpi7wyneddl7iwt
Start Date:  2021-10-31T03:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  6187
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv7h95zf95jl0lpi7wyneddl7iwt
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7h95zc7wnr3vfgj8vwr5a2g41p
Start Date:  2021-10-31T03:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  6287
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7h96ksk2gwy3guchaa6n61xenx
Start Date:  2021-10-31T04:00:00.000Z
# of

-------------------
Token:  b26v89c19zqg8o3fpdv7hdeu6m0nen0vbi5neicrqy5bx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7hdeu3leu8t31btsolc6rwr919
Start Date:  2021-10-31T15:00:00.000Z
# of Tweets added from this response:  99
Total # of Tweets added:  9076
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv7hdeu3leu8t31btsolc6rwr919
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7hdeu2258g13uyrkydfl85w4n1
Start Date:  2021-10-31T15:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  9176
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7hdffldf7fktmhvubdyj28fr0d
Start Date:  2021-10-31T16:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  9276
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv7hdffldf7fktmhvubdyj28fr0d
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7hdffjuddd26awkvy1ykr4szgd
St

-------------------
Token:  b26v89c19zqg8o3fpdv7w1ynsgebvhcwge92lbzbmae7x
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7w1ynsewmb5ou0i9uticmgxgfx
Start Date:  2021-11-01T03:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  12066
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv7w1ynsewmb5ou0i9uticmgxgfx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7w1yddnrrb9reed4jb9c90dabh
Start Date:  2021-11-01T03:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  12166
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7w1yysg8vbbonuwhh7syeigl19
Start Date:  2021-11-01T04:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  12266
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv7w1yysg8vbbonuwhh7syeigl19
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7w1yyqwrupkgje1nrzcyby3qf

...

-------------------
Token:  b26v89c19zqg8o3fpdv7wcoe6nh81rcifz703a646uwe5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7wcoe54v1vjrwopqq8wg4522yl
Start Date:  2021-11-02T17:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  21045
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7weqfh8t0puiq1pf5dkuddxgu5
Start Date:  2021-11-02T18:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  21145
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv7weqfh8t0puiq1pf5dkuddxgu5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7weqffqlsfrtwgtr2va9h4c96l
Start Date:  2021-11-02T18:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  21245
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7wer0uinnqd1n7n2cm99xzy2d9
Start Date:  2021-11-02T19:00:00.000Z
#

-------------------
Token:  b26v89c19zqg8o3fpdv7wgx8rx03v88fmohwbp9kdaf0d
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7wgwyblqxzmsayckzuez3xcfwd
Start Date:  2021-11-03T05:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  24036
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7wgxjxy9l8fa6t5vkb3ey82hrx
Start Date:  2021-11-03T06:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  24136
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv7wgxjxy9l8fa6t5vkb3ey82hrx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7wgxjtfgulzr7p5bx0kcnoapod
Start Date:  2021-11-03T06:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  24236
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7wizvlsqtii1uxihdj2h6j87zx
Start Date:  2021-11-03T07:00:00.000Z
#

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7wl6p42wf82v30c8wflw9sampp
Start Date:  2021-11-03T18:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  27024
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv7wl6p42wf82v30c8wflw9sampp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7wl6ent58lpzl6gztlkhoaoku5
Start Date:  2021-11-03T18:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  27124
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7wl7043e3iu763wciujsqc95kt
Start Date:  2021-11-03T19:00:00.000Z
# of Tweets added from this response:  99
Total # of Tweets added:  27223
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv7wl7043e3iu763wciujsqc95kt
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv7wl7012sfqtu8iyo1j75wau0sd
Start Date:  2021-11-03T19:00:00.000Z
# 

-------------------
Token:  b26v89c19zqg8o3fpdv8b9pxtcx7v7eli4o6byf3zcepp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv8b9pxq9z17l7wvjmf8076b00ot
Start Date:  2021-11-04T06:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  30017
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv8b9qj9mqnl2qwuzuz25o9yhbb1
Start Date:  2021-11-04T07:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  30117
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv8b9qj9mqnl2qwuzuz25o9yhbb1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv8b9qj52nl4vkfxnr09w6343drx
Start Date:  2021-11-04T07:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  30217
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv8b9r4ldyrg4twsnq6qizl9d7y5
Start Date:  2021-11-04T08:00:00.000Z
#

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv8be0ah9woy5te0wyyl24xd06m5
Start Date:  2021-11-04T21:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  33015
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv8be0ah9woy5te0wyyl24xd06m5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv8be0ae9ij895qpkwi0enuw8br1
Start Date:  2021-11-04T21:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  33115
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv8be0vuiwb36pfmhxlrj3qqh8cd
Start Date:  2021-11-04T22:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  33215
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv8be0vuiwb36pfmhxlrj3qqh8cd
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv8be0vriak2in1ie7qw40zstqpp
Start Date:  2021-11-04T22:00:00.000Z
#